<center><img src="/files/images/DLI_Header.png" /></center>

# 随机人格

在此 notebook 中，您将学习如何通过控制模型生成的**温度**（temperature）来控制其响应的随机性。利用**温度**，您将能够创造各种 AI 人格。

## 学习目标

完成此 notebook 后，您将能够：
* 解释**采样**是如何导致非确定性 LLM 响应的。
* 通过调整**温度**来控制响应的随机度。

## 视频教程

执行以下单元以加载此 notebook 的视频教程。

In [ ]:
 from IPython.display import HTML

video_url = "https://d36m44n9vdbmda.cloudfront.net/assets/s-fx-12-v1/v2/05-random.mp4"

video_html = f"""
<video controls width="640" height="360">
    <source src="{video_url}" type="video/mp4">
    Your browser does not support the video tag.
</video>
"""

display(HTML(video_html))

## 创建 LLaMA-2 工作流

In [ ]:
from transformers import pipeline
model = "TheBloke/Llama-2-13B-chat-GPTQ"
# model = "TheBloke/Llama-2-7B-chat-GPTQ"

llama_pipe = pipeline("text-generation", model=model, device_map="auto");

## 辅助函数

在此 notebook 中，我们将使用下面这个函数与 LLM 交互。可以先大概看一眼，后面会详细介绍。

### 生成模型响应

In [ ]:
def generate(prompt, max_length=1024, pipe=llama_pipe, **kwargs):
    """
    Generates a response to the given prompt using a specified language model pipeline.

    This function takes a prompt and passes it to a language model pipeline, such as LLaMA, 
    to generate a text response. The function is designed to allow customization of the 
    generation process through various parameters and keyword arguments.

    Parameters:
    - prompt (str): The input text prompt to generate a response for.
    - max_length (int): The maximum length of the generated response. Default is 1024 tokens.
    - pipe (callable): The language model pipeline function used for generation. Default is llama_pipe.
    - **kwargs: Additional keyword arguments that are passed to the pipeline function.

    Returns:
    - str: The generated text response from the model, trimmed of leading and trailing whitespace.

    Example usage:
    ```
    prompt_text = "Explain the theory of relativity."
    response = generate(prompt_text, max_length=512, pipe=my_custom_pipeline, temperature=0.7)
    print(response)
    ```
    """

    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

### 构造提示词，包括可选的系统上下文和/或示例

In [ ]:
def construct_prompt_with_context(main_prompt, system_context="", conversation_examples=[]):
    """
    Constructs a complete structured prompt for a language model, including optional system context and conversation examples.

    This function compiles a prompt that can be directly used for generating responses from a language model. 
    It creates a structured format that begins with an optional system context message, appends a series of conversational 
    examples as prior interactions, and ends with the main user prompt. If no system context or conversation examples are provided,
    it will return only the main prompt.

    Parameters:
    - main_prompt (str): The core question or statement for the language model to respond to.
    - system_context (str, optional): Additional context or information about the scenario or environment. Defaults to an empty string.
    - conversation_examples (list of tuples, optional): Prior exchanges provided as context, where each tuple contains a user message 
      and a corresponding agent response. Defaults to an empty list.

    Returns:
    - str: A string formatted as a complete prompt ready for language model input. If no system context or examples are provided, returns the main prompt.

    Example usage:
    ```
    main_prompt = "I'm looking to improve my dialogue writing skills for my next short story. Any suggestions?"
    system_context = "User is an aspiring author seeking to enhance dialogue writing techniques."
    conversation_examples = [
        ("How can dialogue contribute to character development?", "Dialogue should reveal character traits and show personal growth over the story arc."),
        ("What are some common pitfalls in writing dialogue?", "Avoid exposition dumps in dialogue and make sure each character's voice is distinct.")
    ]

    full_prompt = construct_prompt_with_context(main_prompt, system_context, conversation_examples)
    print(full_prompt)
    ```
    """
    
    # Return the main prompt if no system context or conversation examples are provided
    if not system_context and not conversation_examples:
        return main_prompt

    # Start with the initial part of the prompt including the system context, if provided
    full_prompt = f"<s>[INST] <<SYS>>{system_context}<</SYS>>\n" if system_context else "<s>[INST]\n"

    # Add each example from the conversation_examples to the prompt
    for user_msg, agent_response in conversation_examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    # Add the main user prompt at the end
    full_prompt += f"{main_prompt} [/INST]"

    return full_prompt

## 默认的非随机响应

您可能已在之前的 notebook 中注意到，用相同的提示词，LLaMA-2 模型就会返回相同的响应。下面我们来观察一下。

这里，我们将使用模型执行另一项生成任务：生成 Galaxy Rider 山地自行车的虚拟客户体验。我们已经设置了恰当的系统上下文，提示模型去回忆一天难忘的骑行体验。

In [ ]:
system_context = """
You are a perennially satisfied customer who loves to reminisce about personal experiences with products. \
You never delve into technical specifics, as you believe it's the emotion and the joy that matter most. \
You're excited for others to feel the same euphoria and happiness you do. Your aim isn't to advertise, \
but to share a genuine, heartfelt story of joy and contentment.
"""

prompt = "Recall a memorable day out with your Galaxy Rider mountain bike in 50 words or less."

In [ ]:
print(generate(construct_prompt_with_context(prompt, system_context)))

---

让我们用完全相同的提示词再次生成一个响应。

In [ ]:
print(generate(construct_prompt_with_context(prompt, system_context)))

In [ ]:
print(generate(construct_prompt_with_context(prompt, system_context)))

---

如上所述，模型每次都会生成完全相同的响应。在许多场景下，这正是我们想要的行为，但对于其它场景，我们想让模型的响应有一定的随机性。为此，我们将修改模型响应的**温度**。

## 采样和温度

在语言模型领域，**采样**是指模型通过从潜在的下一个词（其实是 token，但我们可以把 token 先当成是单词）的概率分布中进行采样的过程。当我们在不启用**采样**的情况下与语言模型交互，模型的运行结果就会是确定的，也就是会稳定的选择可能性最大的下一个词。当您需要一致性和准确性时，这种默认行为很有用，但满足不了创造性和多样化的需求。

用 `transformers` 工作流与 LLaMA-2 模型交互时，默认情况下**采样**是被*禁用*的。在调用 `generator` 的时候设置 `do_sample=True` 就可以开启采样。这样就让模型根据概率分布来选择词语，使得概率较低的词也能被选中，从而生成更多样化、更有趣的文本。

启用采样后，我们还可以传入特定的**温度**值，您可以将其视为响应的随机程度。`temperature` 需要我们传一个 `0.0` 到 `1.0` 之间的值，值越大随机性就越强。

## 练习：随机响应

复用与上面相同的**系统上下文**和提示词，启用**采样**（`do_sample=True`）并将**温度**设为最高值（`temperature=1.0`）。生成 3 种不同的响应，看看它们是不是各不相同。

如果遇到问题，请查看下方的参考答案。

## 您的代码

## 参考答案

In [ ]:
print(generate(construct_prompt_with_context(prompt, system_context), do_sample=True, temperature=1))

In [ ]:
print(generate(construct_prompt_with_context(prompt, system_context), do_sample=True, temperature=1))

In [ ]:
print(generate(construct_prompt_with_context(prompt, system_context), do_sample=True, temperature=1))

## 创造力和准确性

最后，关于 `temperature`，值得一提的是随机生成虽然能满足对独特性和创意的需求，但不具备精确性。在对模型的精确性或事实准确性有要求的情况下，始终记得在增加温度时检查其输出。

## 关键概念回顾

此 notebook 中介绍了以下关键概念：
* **采样**：文本生成过程中，语言模型根据词汇概率分布选择下一个词（token）的过程。
* **温度**：用于控制采样随机性的超参数。温度越高，响应就越具多样性和创造性，温度越低，模型的输出就越能被预测、越保守。

## 可选的进阶练习

如果您想超出本课程的内容进阶一下，可以试试下面的额外开放式练习。

### 使用 7B 模型

在 notebook 顶部，按照下面的代码重启内核后，取消注释以使用 7B 模型，而不是 13B 模型。试试通过提示工程在使用小（更弱）模型的情况下获得满意的结果。

### 制作交互角色

现在，您可以生成虚拟人物的对话，尝试创建一个小型系统，创建多种不同的个性，并让他们产生交互。

### 让可交互角色玩游戏

扩展之前的练习，创建多个致力于实现某个“游戏”所定义目标的角色。可以是试图让一个角色说出某个词，或者是泄露一个秘密的藏宝点，甚至可以是某种让角色携手合作才能实现的目标。如果您真的想挑战一下自己，甚至可以考虑创建两个以上或是一组玩家进行互动。

## 重启内核

为下一个 notebook 释放 GPU 显存，请运行以下单元。

In [ ]:
from IPython import get_ipython

get_ipython().kernel.do_shutdown(restart=True)